# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,298.14,73,98,1.88,PN,1706989348
1,1,las khorey,11.1595,48.1967,298.35,76,0,3.43,SO,1706989348
2,2,waitangi,-43.9535,-176.5597,289.16,84,52,0.45,NZ,1706989348
3,3,arkhangel'sk,64.5401,40.5433,267.21,93,100,4.00,RU,1706989348
4,4,puerto natales,-51.7236,-72.4875,289.40,36,100,10.29,CL,1706989349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    geo=True,
    frame_width=800,
    frame_height=600,
    
    size="Humidity",
    hover_cols=['City']
    
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#Ideal conditions: cloudiness<30, humidity<50
new_df = city_data_df[(city_data_df['Cloudiness']<30) & (city_data_df['Humidity']<50)]
new_df

# Drop any rows with null values
new_df.dropna(inplace=True)

# Display sample data
new_df.head()

C:\Users\schre\AppData\Local\Temp\ipykernel_14708\222531069.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,tazacorte,28.6290,-17.9293,294.69,30,0,3.60,ES,1706989354
41,41,bousso,10.4795,16.7117,298.53,16,4,3.51,TD,1706989354
63,63,rakops,-21.0167,24.3333,300.15,34,2,2.59,BW,1706989357
79,79,hansot,21.5833,72.8000,297.32,44,0,3.29,IN,1706989359
83,83,ilulissat,69.2167,-51.1000,252.16,49,20,4.12,GL,1706989360


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=new_df.copy()
hotel_df=hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df.head()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df[ "Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
35,tazacorte,ES,28.6290,-17.9293,30,
41,bousso,TD,10.4795,16.7117,16,
63,rakops,BW,-21.0167,24.3333,34,
79,hansot,IN,21.5833,72.8000,44,
83,ilulissat,GL,69.2167,-51.1000,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

categories='accommodation.hotel'
limit=5

latitude=0
longitude=0
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

params = {
    "categories":categories,
    "limit":limit,
    "filter":filter,
    "bias":bias,
    "apiKey":geoapify_key    
    }



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tazacorte - nearest hotel: App Leyma
bousso - nearest hotel: No hotel found
rakops - nearest hotel: Rakops River Lodge
hansot - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
awjilah - nearest hotel: No hotel found
kissidougou - nearest hotel: Mantisse
al kharijah - nearest hotel: Kharga Hotel
tindouf - nearest hotel: محمد بوسبي
rikaze - nearest hotel: Gesar Hotel
toledo - nearest hotel: Cigarral Del Pintor
fada - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
ati - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
laayoune - nearest hotel: Hôtel Lakouara
palmer - nearest hotel: No hotel found
aoulef - nearest hotel: دار الضياف
challapata - nearest hotel: Residencial Marian
rapar - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
fort bragg - nearest hotel: Airborne Inn Lodging
calingasta - nearest hotel: Nora 

,City,Country,Lat,Lng,Humidity,Hotel Name
35,tazacorte,ES,28.6290,-17.9293,30,App Leyma
41,bousso,TD,10.4795,16.7117,16,No hotel found
63,rakops,BW,-21.0167,24.3333,34,Rakops River Lodge
79,hansot,IN,21.5833,72.8000,44,No hotel found
83,ilulissat,GL,69.2167,-51.1000,49,Best Western Plus Hotel Ilulissat
91,awjilah,LY,29.1081,21.2869,34,No hotel found
92,kissidougou,GN,9.1848,-10.0999,19,Mantisse
106,al kharijah,EG,25.4514,30.5464,34,Kharga Hotel
110,tindouf,DZ,27.6711,-8.1474,9,محمد بوسبي
114,rikaze,CN,29.2500,88.8833,16,Gesar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:


# hover_cols argument in hvplot not working          

# Configure the map plot_2
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    geo=True,
    frame_width=800,
    frame_height=600,
    alpha=0.80,
    size="Humidity",
    color='City',
    hover_cols=['Hotel Name' ,'Country']
    
    
)

# Display the map plot_2
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)